In [8]:
import pandas as pd
import polars as pl
pd.set_option('display.max_colwidth', 70) # чтобы влезали хеши


import numpy as np
import math
from decimal import Decimal
import pprint
from datetime import datetime, timedelta
import time
import json




import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import copy
import matplotlib.pyplot as plt
from tqdm import tqdm


import sys
import os


sys.path.append(os.path.split(os.getcwd())[0])


from strategy.primitives import Fee, Token, Pool, TOKEN_DETAILS, POOLS, Token, SPACING
from strategy.data import PoolDataUniV3, RawDataUniV3, SyntheticData, DownloaderRawDataUniV3
from strategy.cross_validation import FolderSimple, CrossValidation


from strategy.strategies import AbstractStrategy, Hold, UniV3Passive
from strategy.portfolio import Portfolio
from strategy.backtest import Backtest
from strategy.positions import BiCurrencyPosition, UniV3Position
from strategy.viewers import PotrfolioViewer, UniswapViewer, RebalanceViewer, LiquidityViewer

from utilities.utilities import ConfigParser, get_db_connector, get_main_path

In [9]:
pool_num = 1
pool = Pool(POOLS[pool_num]['token0'], POOLS[pool_num]['token1'], POOLS[pool_num]['fee'])

# Binance

In [10]:
# https://python-binance.readthedocs.io/en/latest/
from binance import Client

In [11]:
from utilities.utilities import get_data_from_binance
df = get_data_from_binance(
    'ETHBTC',
    '1w',
    '01-12-2010',
    '01-03-2022'
)

start: 2022-03-18 14:32:35.312261
finish: 2022-03-18 14:32:35.801551
rows downloaded: 235
mismatch timestamp rows:  0
df shape (234, 2) saved to /root/aenik_ws/mellow-strategy-sdk/data/ETHBTC_1w_01-12-2010_01-03-2022.csv


In [12]:
df.head(3)

,timestamp,price
0,2017-07-17,0.082241
1,2017-07-24,0.083533
2,2017-07-31,0.070109


In [ ]:
intervals_fast = ['1h', '2h', '4h', '6h', '8h', '12h', '1d', '3d', '1w']
intervals_slow = ['1m', '3m', '5m', '15m', '30m']

pairs = ['ETHBTC', 'ETHUSDC', 'USDCUSDT', 'WBTCETH']

for pair in pairs:
    for interval in intervals_fast:
        df = get_data_from_binance(
            pair,
            interval,
            '01-12-2010',
            '01-03-2022'
        )

In [ ]:
for pair in pairs:
    df = pd.read_csv(f'../data/{pair}_1w_01-12-2010_01-03-2022.csv', parse_dates=False)
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S:%f')
    df.set_index('timestamp').plot(title=pair)


# SyntheticData

In [18]:
synth_data = SyntheticData(
    pool=pool, 
    start_date='1-1-2022', 
    n_points=365, 
    init_price=1, 
    mu=0, 
    sigma=0.1, 
    seed=42
).generate_data()

In [25]:
synth_data.mints is None, synth_data.burns is None, synth_data.swaps.shape

(True, True, (365, 4))

# Pool data

In [27]:
from strategy.data import DownloaderRawDataUniV3
# ~5min, only the first time
DownloaderRawDataUniV3().load_events(pool_num)
# ~5min, only the first time
# DownloaderRawDataUniV3().get_transactions()

get burn
saved to burn_WBTC_WETH_3000.csv
get mint
saved to mint_WBTC_WETH_3000.csv
get swap
saved to swap_WBTC_WETH_3000.csv


In [29]:
from strategy.data import RawDataUniV3

data = RawDataUniV3(pool).load_from_folder()

In [31]:
data.mints.shape, data.burns.shape, data.swaps.shape

((11262, 11), (8307, 11), (64101, 15))